In [ ]:
!git clone "https://github.com/RuslanZaripov/rhtr.git"

# Move to the root folder

In [2]:
import os

os.getcwd()

'C:\\Users\\rusla\\Desktop\\ITMO\\7-sem\\thesis\\rhtr\\src\\segmentation\\scripts'

In [17]:
import os

os.chdir("../../../")
os.getcwd()

'C:\\Users\\rusla\\Desktop\\ITMO\\7-sem\\thesis\\rhtr'

# Download the datasets

In [22]:
data_folder = 'data/raw/'

In [26]:
dataset_name = 'school_notebooks_EN'
dataset_path = data_folder + dataset_name

In [23]:
!rm -rf /kaggle/working/school_notebooks_EN
!git lfs install
!git clone https://huggingface.co/datasets/ai-forever/school_notebooks_EN {dataset_path}

Updated Git hooks.
Git LFS initialized.


Cloning into 'data/raw/school_notebooks_EN'...


In [ ]:
!unzip -o -q {dataset_path + '/images.zip'} -d {dataset_path + '/school_notebooks_EN'} > /dev/null

In [27]:
dataset_name = 'school_notebooks_RU'
dataset_path = data_folder + dataset_name

In [6]:
!rm -rf /kaggle/working/school_notebooks_RU
!git lfs install
!git clone https://huggingface.co/datasets/ai-forever/school_notebooks_RU {dataset_path}

/c/Users/rusla/Desktop/ITMO/7-sem/thesis/rhtr/src/segmentation/scripts


In [ ]:
!rm -rf /kaggle/working/school_notebooks_RU
!git lfs install
!git clone https://huggingface.co/datasets/ai-forever/school_notebooks_RU {data_path}

In [ ]:
!unzip -o -q {dataset_path + '/images.zip'} -d {dataset_path + '/school_notebooks_RU'} > /dev/null

In [ ]:
import albumentations as A
from src.segmentation.preprocessing import *
import numpy as np


def preprocessing(input_image):
    # gray = grayscale(input_image)
    # cl = clahe1channel(gray)
    # g = gaussian(cl)
    b = bilateral(input_image)
    return b


ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

validation_transform = A.Compose([
    A.Normalize(),
    A.Resize(width=768, height=768),
])

train_transform = A.Compose([
    A.Normalize(),
    A.Resize(width=768, height=768),
])

In [ ]:
from src.segmentation.dataset import SchoolSegmentationDataset

validation_set_ru = SchoolSegmentationDataset(
    '/kaggle/working/school_notebooks_RU/annotations_val.json',
    '/kaggle/working/school_notebooks_RU/images',
    folder_name='SchoolSegmentationDatasetRU',
    preprocessing=preprocessing,
    transform=validation_transform,
    df_path=None,
    keep=True)

In [ ]:
from src.segmentation.dataset import dynamic_length_collate
from torch.utils.data import Dataset
import torch

validation_loader = torch.utils.data.DataLoader(
    validation_set_ru,
    batch_size=4,
    shuffle=False,
    collate_fn=dynamic_length_collate)

In [31]:
from src.segmentation.metrics import AverageMeter
from src.segmentation.utils import visualize, background_ratio
from tqdm import tqdm

interval = 10
pos_to_show = 0

# black_pixels_thresh = AverageMeter()
black_pixels_binary = AverageMeter('black_pixels_binary')
black_pixels_lines = AverageMeter('black_pixels_lines')

for idx, data in tqdm(enumerate(validation_loader), total=len(validation_loader)):
    #     print(f"{data[0][pos_to_show].min()}-{data[0][pos_to_show].max()}")

    #     print(f"{data[0].shape=}")
    #     print(f"{data[0][0].shape=}")
    #     print(f"{data[0][0].min()}-{data[0][0].max()}")
    original = np.transpose(data[0][pos_to_show], (1, 2, 0))

    #     print(f"{type(data[1])=}")
    #     print(f"{data[1]['watershed_masks'].shape=}")
    #     print(f"{data[1]['binary'].shape=}")
    #     print(f"{data[1]['thresh'].shape=}")

    #     thresh = data[1]['thresh'][0].squeeze(0)
    #     instance = data[1]['instance'][0].squeeze(0)
    watershed_first = data[1]['watershed'][0][0]
    watershed_last = data[1]['watershed'][0][-1]

    binary = data[1]['binary'][pos_to_show].squeeze(0)
    lines = data[1]['lines'][pos_to_show].squeeze(0)
    borders = data[1]['border_mask'][pos_to_show].squeeze(0)
    word_polygons = data[1]['word_polygons'][pos_to_show]

    if idx % interval == (interval - 1):
        print(f"{data[1].keys()=}")

        #         print(f"{background_ratio(thresh)=}")
        print(f"{background_ratio(binary)=}")
        print(f"{background_ratio(lines)=}")
        print()

        visualize(original, 'original', cmap='gray')
        visualize(binary, 'binary')
        visualize(lines, 'lines')
        visualize(borders, 'borders')
        #         visualize(instance, 'instance')
        #         visualize(thresh, 'thresh')
        visualize(watershed_first, 'watershed_masks_0')
        visualize(watershed_last, 'watershed_masks_last')

        import numpy as np
        import matplotlib.pyplot as plt
        import matplotlib.patches as patches

        image = np.zeros((768, 768), dtype=np.uint8)

        fig, ax = plt.subplots()
        ax.imshow(image)

        for polygon in word_polygons:
            from shapely.geometry import Polygon

            if len(polygon) == 0: continue
            poly = patches.Polygon(polygon, closed=True, edgecolor='r', alpha=0.5)
            ax.add_patch(poly)

        plt.axis('off')
        plt.show()

    #     black_pixels_thresh.update(background_ratio(thresh))
    black_pixels_binary.update(background_ratio(binary))
    black_pixels_lines.update(background_ratio(lines))

# print(f"{black_pixels_thresh.avg=}")
print(f"{black_pixels_lines.avg=}")
print(f"{black_pixels_binary.avg=}")

NameError: name 'AverageMeter' is not defined